<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:

------


In [2]:
from edc import check_compatibility
check_compatibility("v0.18.4")

Notebook is compatible


## xcube Generator Python Access Library

This notebook shows how to generate xcube cube using the xcube-gen web service [xcube-gen.brockmann-consult.de](https://xcube-gen.brockmann-consult.de).
Please be aware, this notebook will not run unless you have access to the xcube-gen service as well as a bucket on AWS.

In [3]:
from jobapi import JobApi
api = JobApi()
api.whoami

ModuleNotFoundError: No module named 'xcube_gen'

## Generate a config

In [ ]:
import os
cfg = {
  "input_configs": [
    {
      "store_id": "@sentinelhub",
      "data_id": "S2L2A",
      "open_params": {
        "tile_size": [
          1000,
          1000
        ]
      }
    }
  ],
  "cube_config": {
    "variable_names": [
      "B01",
      "B02"
    ],
    "bbox": [
      7,
      53,
      9,
      55
    ],
    "spatial_res": 0.001,
    "crs": "WGS84",
    "time_range": [
      "2000-06-20",
      "2000-06-22"
    ],
    "time_period": "1D"
  },
  "output_config": {
    "store_id": "s3",
    "store_params": {
      "bucket_name": os.environ["AWS_BUCKET"],
      "aws_access_key_id": os.environ["AWS_ACCESS_KEY_ID"],
      "aws_secret_access_key": os.environ["AWS_SECRET_ACCESS_KEY"],
    }
  }
}

## Generating an xcube

In [ ]:
api.create(cfg=cfg)

## Getting the Status of a Generation Job

In [ ]:
api.status('xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017')

## Listing my Jobs

In [ ]:
api.list()

## Deleting a job

In [ ]:
api.delete('xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017')